In [1]:
# Importe les librairies utiles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 

# Chemin vers les csv
pathTest = "./test3.csv"
pathTrain = "./train3.csv"

# Importe les jeux de donnees
df_test = pd.read_csv(pathTest, sep=',', index_col=0)
df_train = pd.read_csv(pathTrain, sep=',', index_col=0)

# On enleve tout de suite la colonne Name de notre jeu de donnees train
df_train = df_train.drop(['Name'], axis=1)



# --- Variables Explicatives ---

#    Name : Nom du jeu
#    Platform : Console sur laquelle le jeu fonctionne
#    Year_of_Release : Année de sortie du jeu
#    Genre
#    Publisher
#    JP_Sales : Nombre de ventes du jeu au Japon en millions d’unités
#    Other_Sales : Nombre de ventes du jeu ailleurs dans le monde : Afrique, Asie sans le Japon, Europe sans l’Union Européenne et Amérique du Sud en millions d’unités
#    Critic_Score : Score donné par Metacritic
#    Critic_Count : Nombre de critiques prises en compte pour estimer le Critic_score
#    User_Score : Score donné par les usagers de Metacritic
#    User_Count : Nombre d’usagers considérés pour estimer le User_Score
#    Developer : Compagnie créatrice du jeu
#    Rating : Classement ESRB (Entertainment Software Rating Board) ie à qui s’addresse le jeu (tout public, majeur, adolescents, etc) 


# --- Variables d'Interet ---

#    NA_sales : Nombre de ventes du jeu en Amérique du Nord en millions d’unités
#    Global_Sales : Nombre de ventes total du jeu en millions d’unités

In [2]:
"""
--- Evaluation des predictions --- 

Dans le document MTH3302_CriteresProjet-1.pdf on nous informe que la precision
de nos estimations sera evaluee avec le root mean square error (RMSE)

On definit cette fonction ci-bas,

    Y : Variable d'interet
    W : Predictions de la variable d'interet

"""

def RMSE(Y,W):
    
    # Nombre d'observations
    n = len(Y)
    
    total = 0.0
    for i in range(n):
        total += (Y[i] - W[i])**2
    
    RMSE = total/float(n)
    
    return RMSE
    

In [3]:
"""
--- Variable Qualitative -> Table Binaire --- 

Afin de traiter nos variables qualitatives, nous avons une fonction qui la tranforme en table binaire 
avec un one bit encoder. 

    Disons que la variable explicative peut prendre 4 valeurs {Bleu, Vert, Jaune, Rouge},
        
          ID        X1
        ----------------
          1        Bleu
          2        Bleu
          3        Vert
          4        Jaune
          5        Vert
          6        Rouge          
         ...
    
    Suite a un encodage en 1 bit notre variable explicative X1 devient,
    
          ID        Bleu        Vert        Jaune        Rouge
        -------------------------------------------------------
          1          1           0            0            0
          2          1           0            0            0
          3          0           1            0            0
          4          0           0            1            0
          5          0           1            0            0
          6          0           0            0            1
                                ...
                                

Nous nous basons sur la fonction get_dummies() de la librairie Panda pour faire ceci.



Input,

    trainSet : le jeu de donnees de training
    testSet : le jeu de donnees de test
    varName : la variable qualitative que nous souhaitons transformer en tableau binaire

"""

def qualiToBinary(trainSet, testSet, varName):
    
    # Genere la table binaire de la variable qualitative
    trainVar_binary = pd.get_dummies(trainSet[varName])
    testVar_binary = pd.get_dummies(testSet[varName])
    
    # On enleve la variable qualitative de nos jeux de donnees
    trainSet = trainSet.drop([varName], axis=1)
    testSet = testSet.drop([varName], axis=1)
    
    # On convertit nos tableaux binaires en Dataframe
    trainVar_binary = pd.DataFrame(trainVar_binary)
    testVar_binary = pd.DataFrame(testVar_binary)

    # On ajoute les nouvelles colonnes a nos jeux de donnees
    trainSet = pd.concat([trainSet,trainVar_binary],axis=1)
    testSet = pd.concat([testSet,testVar_binary],axis=1)
    
    # On retourne les jeux de donnees
    return trainSet, testSet

In [4]:
# On peut ici enlever les variables quantitatives non probantes
#df_train_quanti = df_train_quanti.drop(['Year_of_Release'], axis=1)
#df_train_quanti = df_train_quanti.drop(['JP_Sales'], axis=1)
#df_train_quanti = df_train_quanti.drop(['Other_Sales'], axis=1)
#df_train_quanti = df_train_quanti.drop(['Critic_Score'], axis=1)
#df_train_quanti = df_train_quanti.drop(['Critic_Count'], axis=1)
#df_train_quanti = df_train_quanti.drop(['User_Score'], axis=1)
#df_train_quanti = df_train_quanti.drop(['User_Count'], axis=1)
#df_train_quanti = df_train_quanti.drop(['NA_Sales'], axis=1)
#df_train_quanti = df_train_quanti.drop(['Global_Sales'], axis=1)

#df_test_quanti = df_test_quanti.drop(['Year_of_Release'], axis=1)
#df_test_quanti = df_test_quanti.drop(['JP_Sales'], axis=1)
#df_test_quanti = df_test_quanti.drop(['Other_Sales'], axis=1)
#df_test_quanti = df_test_quanti.drop(['Critic_Score'], axis=1)
#df_test_quanti = df_test_quanti.drop(['Critic_Count'], axis=1)
#df_test_quanti = df_test_quanti.drop(['User_Score'], axis=1)
#df_test_quanti = df_test_quanti.drop(['User_Count'], axis=1)

In [5]:
# Variable Qualitative #1 : Platform
print("Nbr. of categories (train) : %d" %(df_train.Platform.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Platform.nunique()))

df_train, df_test = qualiToBinary(df_train, df_test, 'Platform')

Nbr. of categories (train) : 30
Nbr. of categories (test) : 27


In [6]:
# Variable Qualitative #2 : Genre
print("Nbr. of categories (train) : %d" %(df_train.Genre.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Genre.nunique()))

df_train, df_test = qualiToBinary(df_train, df_test,'Genre')

Nbr. of categories (train) : 12
Nbr. of categories (test) : 12


In [7]:
# Variable Qualitative #3 : Publisher
print("Nbr. of categories (train) : %d" %(df_train.Publisher.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Publisher.nunique()))

df_train, df_test = qualiToBinary(df_train, df_test, 'Publisher')

Nbr. of categories (train) : 552
Nbr. of categories (test) : 280


In [8]:
# Variable Qualitative #4 : Developer
print("Nbr. of categories (train) : %d" %(df_train.Developer.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Developer.nunique()))


# Bon ici on a un probleme, si on ajoute les colonnes binaires generees par cette variable on ajoute 100mb
# a notre csv. Ceci rend peu pratique le prototypage, pour l'instant on le laisse tomber. On essayera de ce
# donner une raison rationnelle de le domper plus tard dans l'analyse. Hypothese, beaucoup de colinearite avec 
# le Publisher.

#df_train, df_test = qualiToBinary(df_train, df_test,'Developer')
df_test = df_test.drop(['Developer'],axis=1)
df_train = df_train.drop(['Developer'],axis=1)

Nbr. of categories (train) : 1593
Nbr. of categories (test) : 677


In [9]:
# Variable Qualitative #5 : Rating
print("Nbr. of categories (train) : %d" %(df_train.Rating.nunique()))
print("Nbr. of categories (test) : %d" %(df_test.Rating.nunique()))

df_train, df_test = qualiToBinary(df_train, df_test,'Rating')

Nbr. of categories (train) : 8
Nbr. of categories (test) : 5


In [10]:
# On identifie les colonnes manquantes dans le jeu de donnees test
missing_categories = set(df_train) - set(df_test)

# On ajoute ces colonnes manquantes avec une valeur par defaut de 0
for c in missing_categories:
    df_test[c] = 0

# On identifie les colonnes manquantes dans le jeu de donnees training
missing_categories = set(df_test) - set(df_train)

# On ajoute ces colonnes manquantes avec une valeur par defaut de 0
for c in missing_categories:
    df_train[c] = 0


# On aligne nos jeux de donnees pour que les memes colonnes soient aux memes endroits
df_train, df_test = df_train.align(df_test, axis=1)

In [11]:
# On regarde le nombre de NaN par variables explicatives
print("Nombre de NaN sur %d observations \n" % (len(df_train)))

counter = 0;
for i in df_train.isna().sum():
    if(i > 0):
        print("%s : %d" % (df_train.columns[counter],i))
        
    counter += 1

Nombre de NaN sur 14094 observations 

Critic_Count : 7186
Critic_Score : 7186
User_Count : 7653
User_Score : 7653
Year_of_Release : 232


In [12]:
# On garde seulement les observations ou nous avons toutes les donnees (a ameliorer plus tard)
df_train = df_train.dropna()

# On calcule le nombre d'observations restantes
print("\nApres clean-up, nombre d'observation : %d" % (len(df_train)))


Apres clean-up, nombre d'observation : 5857


In [13]:
# On calcule le nombre de colonne
print("\nNombre de colonne: %d" % (len(df_train.T)))

# On enleve les colonnes qui ne comportent que des zeros
cols_only0 = (df_train != 0).any(axis=0)
df_train = df_train.loc[:,cols_only0]
df_test = df_test.loc[:,cols_only0]

# On enleve toutes les colonnes qui sont en double
duplicates = df_train.T.duplicated()
duplicates = ~duplicates
df_train = df_train.T[duplicates].T
df_test = df_test.T[duplicates].T

# On calcule le nombre de colonne
print("\nNombre de colonne apres clean up : %d" % (len(df_train.T)))


Nombre de colonne: 641

Nombre de colonne apres clean up : 296


In [14]:
# On sauvegarde nos deux nouveaux jeux de donnees entierement quantitatif
#df_train.to_csv("train_clean.csv")
#df_test.to_csv("test_clean.csv")

In [15]:
def linear_regression(y,x):
    
    # Number of Explicative Variables
    k = 0
    try:
        k = x.shape[1]
    except IndexError:
        k = 1
        
    
    # The variance-covariance
    C = np.linalg.inv(np.dot(x.T,x))
    
    B = np.dot(np.dot(C,x.T),y)
    
    return B, np.dot(x,B)

In [16]:
def R2_adj(Y,W,p):

    SS_tot = 0.0
    SS_reg = 0.0
    SS_res = 0.0    
    
    y_S = np.sum(Y)/len(Y)

    for i in range(len(Y)):
        SS_tot += (Y[i] - y_S)**2
        SS_reg += (W[i] - y_S)**2
        SS_res += (Y[i] - W[i])**2
        
    
    n = len(Y)
    
    Radj = 1 - ((SS_res)/float(n-p))/((SS_tot)/(n-1))

    return Radj

In [17]:
def PCA(Y,X,varExplained):
    
    # On transforme X en une matrice de moyenne = 0 et variance = 1 
    X_std = StandardScaler().fit_transform(X)
    nbrOfVar = X.shape[1]

    # On calcule la matrice de covariance
    E = np.dot(X_std.T,X_std)/(nbrOfVar-1)

    # On calcule nos Valeurs et Vecteurs propres
    eig_val, eig_vec = np.linalg.eig(E)
    eig_val = eig_val.real
    eig_vec = eig_vec.real
    eig_vec = eig_vec.T # pour les ordonner par range

    # On calcule la contribution de chaque valeur propre a la variance total
    eig_val_contribution = eig_val/np.sum(eig_val)

    # On cree une liste d'index de 0 au nombre de valeurs propres
    rangeIndex = np.array(range(len(eig_val_contribution)))

    # On cree un tableau indexe de nos valeurs propres et leur contribution a la variance
    eig_val_table = np.array(([rangeIndex,eig_val,eig_val_contribution])).T

    # On ordonne en ordre decroissant de contribution le tableau
    eig_val_table_ordered = eig_val_table[(-eig_val_table[:,2]).argsort()]
    
    # On rejoint la quantite de variance souhaitee
    varCumul = 0.0
    varIndex = 0
    for val in eig_val_table_ordered:
        varIndex += 1
        varCumul += val[2]
        if(varCumul >= varExplained):
            break
    
    # On recupere les index des valeurs propres que nous garderont
    indexKept = eig_val_table_ordered[:,0][:varIndex]

    # Our reduced regressor
    K_vect = eig_vec[indexKept.astype(int),:].T

    # Our principal components
    Z = np.dot(X_std,K_vect)

    # Our Regression Coefficiants
    B_partial = np.dot(np.linalg.inv(np.dot(Z.T,Z)),Z.T)
    B = np.dot(B_partial,Y)

    # We calculate the intercept
    uY = np.mean(Y)

    # We add the intercept
    Y_new = np.dot(Z,B) + uY

    # Clip to min 0
    Y_new = Y_new.clip(min=0)

    print("R2 : %.4f" % (R2_adj(Y,Y_new,varIndex)))
    print("RMSE = %.4f" % (RMSE(Y,Y_new)))
    
    return Y_new, B, K_vect, uY


In [18]:
def PCA_trained(X,B,K_vect,uY):
    
    X_std = StandardScaler().fit_transform(X)
    
    # Our principal components
    Z = np.dot(X_std,K_vect)
    
    # We add the intercept
    Y_new = np.dot(Z,B) + uY
    
    # Clip to min 0
    Y_new = Y_new.clip(min=0)

    return Y_new
    

In [19]:
df_test = df_test.fillna(df_test.mean())

In [20]:
# On regarde le nombre de NaN par variables explicatives
print("Nombre de NaN sur %d observations \n" % (len(df_test)))

counter = 0;
for i in df_test.isna().sum():
    if(i > 0):
        print("%s : %d" % (df_test.columns[counter],i))
        
    counter += 1

Nombre de NaN sur 2490 observations 



In [26]:
def addColumn(trainSet,testSet,colName):
    
    # On definit notre variable d'interet
    Y = np.array(trainSet[colName])

    # On definit notre vecteur de variables explicatives
    X = np.array(trainSet.drop([colName], axis=1))

    # We train our model
    [Y_new, B, K_vect, uY] = PCA(Y,X,0.9)
    
    X_test = np.array(testSet.drop([colName], axis=1))    
    Y_test = PCA_trained(X_test,B,K_vect,uY)
    
    return Y_test

In [22]:
df_test['NA_Sales'] = addColumn(df_train,df_test,'NA_Sales')

R2 : 0.7788
RMSE = 0.2038


In [23]:
df_test['Global_Sales'] = addColumn(df_train,df_test,'Global_Sales')

R2 : 0.9271
RMSE = 0.2730


In [24]:
df_test_final = df_test[['Global_Sales','NA_Sales']]

In [25]:
df_test_final.to_csv("test_final.csv")